# FDR

In [ ]:
import numpy as np
from functions import *
from tqdm.auto import tqdm
import ray

In [ ]:
ray.init()

In [ ]:
p_H1 = .1
SR0 = 0
SR1 = .5
N = 1000
T = 50
name = 'severe'
#name = 'gaussian'
q = .25

@ray.remote
def f(T, name): 
    X0 = generate_non_gaussian_data(
        nr = T,
        nc = int( N * (1-p_H1) ), 
        SR0 = 0,
        name = name,
    )
    X1 = generate_non_gaussian_data(
        nr = T,
        nc = int( N * p_H1 ),
        SR0 = SR1,
        name = name,
    )
    X = np.concatenate([X0, X1], axis=1)

    gamma3 = scipy.stats.skew(X.flatten())                    # Skewness
    gamma4 = scipy.stats.kurtosis(X.flatten(), fisher=False)  # Kurtosis (not excess kurtosis)
        
    alpha_, beta_, SR_c, q_hat = control_for_FDR( q, SR0 = SR0, SR1 = SR1, p_H1 = p_H1, T = T, gamma3 = gamma3, gamma4 = gamma4 )

    SRs0 = X0.mean(axis=0) / X0.std(axis=0)
    SRs1 = X1.mean(axis=0) / X1.std(axis=0)
    FP = np.sum( SRs0 > SR_c )
    TP = np.sum( SRs1 > SR_c )
    return FP / (FP + TP)

FDRs = {}
for name in ['gaussian', 'severe']:
    for T in [50,200,1000]:
        FDRs[name, T] = ray.get([f.remote(T, name) for _ in range(1000)])

In [ ]:
fig, axs = plt.subplots(2, 3, figsize = (12, 6), layout = 'constrained')
for i, name in enumerate(['gaussian', 'severe']):
    for j, T in enumerate([50,200,1000]):
        ax = axs[i, j]
        ax.hist(
            FDRs[name, T], 
            facecolor = 'lightblue',  # edgecolor = 'tab:blue', 
            density = True, 
            bins = 20,
        )
        ax.axvline( q, color = 'black', linestyle = '--' )
        ax.set_xlabel('FDP = FP / (FP + TP)')
        ax.set_title('FDP Distribution, q = 0.25\n' f'{name} {T=}')
        for side in ['left', 'right', 'top']:
            ax.spines[side].set_visible(False)
        ax.set_yticks([])
        ax.set_xlim( 0, .5 )
plt.show()